In [1]:
from datetime import datetime
import pathlib
import os
import requests
import gc
import json
import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from tqdm import tqdm
from typing import Dict, Set
from questionanswering import squad
from questionanswering.squad.eval20 import compute_exact, compute_f1

In [2]:
pretrained_map = {
    "albert": "../pretrained/albert-base-v2",
    "electra_small": "../pretrained/google/electra-small-discriminator",
    "electra": "../pretrained/google/electra-base-discriminator",
    "electra_large": "../pretrained/google/electra-large-discriminator",
    "distilroberta": "../pretrained/distilroberta-base",
    "mpnet": "../pretrained/microsoft/mpnet-base",
    "roberta": "../pretrained/roberta-base",
    "roberta_large": "../pretrained/roberta-large",
}
model_name = "electra_small"
ts = "20210918_151710"
model_dir = f"../models/{model_name}/{ts}/lightning_logs/version_0/checkpoints"
pretrained_dir = pretrained_map[model_name]
model_max_length = 512
batch_size = 128
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
pl.seed_everything(31)

31

In [3]:
device = torch.device('cuda:0')
print(f'Using device:{device}')

#Additional Info when using cuda
if device.type == 'cuda':
    for i in range(torch.cuda.device_count()):
        print(f"{i}: {torch.cuda.get_device_name(i)}")
        print('Memory Allocated:\t', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Memory Cached:\t\t', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')

Using device:cuda:0
0: NVIDIA GeForce GTX 1060 6GB
Memory Allocated:	 0.0 GB
Memory Cached:		 0.0 GB


In [4]:
job_dir = f"output/{model_name}"
pathlib.Path(job_dir).mkdir(parents=True, exist_ok=True)
print(f"job_dir={job_dir}")

job_dir=output/electra_small


In [6]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=model_max_length)
print(f"{repr(tokenizer)}\n{tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='../pretrained/google/electra-small-discriminator', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
['input_ids', 'token_type_ids', 'attention_mask']


In [7]:
model = AutoModelForQuestionAnswering.from_pretrained(model_dir)
print(repr(model.config))

ElectraConfig {
  "_name_or_path": "../models/electra_small/20210918_151710/lightning_logs/version_0/checkpoints",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "vocab_size": 30522
}



In [8]:
df = pd.read_parquet("input/dev.parquet")
df.drop_duplicates(subset=["id"], keep='first', inplace=True, ignore_index=True)
qids = list(df["id"])
questions = list(df["question"])
contexts = list(df["context"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11873 entries, 0 to 11872
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             11873 non-null  object
 1   is_impossible  11873 non-null  int8  
 2   title          11873 non-null  object
 3   question       11873 non-null  object
 4   answer_start   11873 non-null  int16 
 5   answer_end     11873 non-null  int16 
 6   answer_text    11873 non-null  object
 7   context        11873 non-null  object
 8   cq_length      11873 non-null  int16 
 9   cq_wp_length   11873 non-null  int16 
 10  cq_sp_length   11873 non-null  int16 
 11  cq_bpe_length  11873 non-null  int16 
dtypes: int16(6), int8(1), object(5)
memory usage: 614.6+ KB


In [9]:
%%time
x = tokenizer(
    questions,
    contexts,
    truncation="only_second",
    padding="max_length",
    return_special_tokens_mask=True,
)

Wall time: 6.06 s


In [10]:
input_ids = x["input_ids"]
special_tokens_mask = torch.tensor(x["special_tokens_mask"])
del x["special_tokens_mask"]
batches = torch.utils.data.DataLoader(squad.Dataset(x), batch_size=batch_size, shuffle=False)
model.eval()
model.to(device)
start_logits = None
end_logits = None
with torch.no_grad():
    for batch in tqdm(batches):
        for k, v in batch.items():
            batch[k] = v.to(device)
        outputs = model(**batch)
        sl = outputs.start_logits
        el = outputs.end_logits
        if start_logits is None:
            start_logits = sl
        else:
            start_logits = torch.cat((start_logits, sl), 0)
        if end_logits is None:
            end_logits = el
        else:
            end_logits = torch.cat((end_logits, el), 0)
print(f"start_logits={start_logits.size()}, end_logits={end_logits.size()}")

100%|███████████████████████████████████| 93/93 [01:43<00:00,  1.11s/it]

start_logits=torch.Size([11873, 512]), end_logits=torch.Size([11873, 512])


In [11]:
%%time
for k in tqdm(range(len(start_logits))):
    qid = qids[k]
    text = ""
    i = int(torch.argmax(start_logits[k]))
    j = int(torch.argmax(end_logits[k])) + 1
    start_score: float = start_logits[k][i].item()
    end_score: float = end_logits[k][j - 1].item()
    if squad.is_valid_answer(
        i,
        j,
        start_score=start_score,
        end_score=end_score,
        special_tokens_mask=special_tokens_mask[k],
    ):
        _ids = input_ids[k][i:j]
        tokens = tokenizer.convert_ids_to_tokens(_ids)
        text = tokenizer.convert_tokens_to_string(tokens)
    res[qid] = text
print(f"len(res)={len(res)}")
is_complete = len(questions) == len(res)

100%|███████████████████████████| 11873/11873 [00:04<00:00, 2642.47it/s]

len(res)=11873
Wall time: 4.49 s


In [12]:
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
filepath = f"{job_dir}/{ts}.json"
os.environ["SQUAD20_PREDICTIONS"] = filepath
with open(filepath, "w") as f:
    json.dump(res, f)
print(f"Saved {filepath}")

Saved output/electra_small/20210918_155652.json


Unix uses `$SQUAD20_PREDICTIONS`

Windows uses `%SQUAD20_PREDICTIONS%`

In [22]:
!python -m questionanswering.squad.eval20 input/dev-v2.0.json %SQUAD20_PREDICTIONS%

{
  "exact": 28.27423566074286,
  "f1": 29.940192700068955,
  "total": 11873,
  "HasAns_exact": 0.016869095816464237,
  "HasAns_f1": 3.353560716585812,
  "HasAns_total": 5928,
  "NoAns_exact": 56.45079899074853,
  "NoAns_f1": 56.45079899074853,
  "NoAns_total": 5945
}


In [14]:
def get_preds(preds):
    def fn(row) -> str:
        qid = row["id"]
        return preds[qid]
    
    return fn


def exact_match(row):
    y_pred = row["pred"]
    y_true = ""
    if row["is_impossible"] == 0:
        y_true = row["answer_text"]
    return compute_exact(y_true, y_pred)


def f1(row):
    y_pred = row["pred"]
    y_true = ""
    if row["is_impossible"] == 0:
        y_true = row["answer_text"]
    return compute_f1(y_true, y_pred)

In [15]:
df["pred"] = df.progress_apply(get_preds(res), axis=1)

100%|█████████████████████████| 11873/11873 [00:00<00:00, 111477.67it/s]


In [16]:
col = "exact_match"
df[col] = df.progress_apply(exact_match, axis=1)
df[col] = df[col].astype(np.int8)

100%|██████████████████████████| 11873/11873 [00:00<00:00, 24425.04it/s]


In [17]:
col = "f1"
df[col] = df.progress_apply(f1, axis=1)
df[col] = df[col].astype(np.float32)

100%|██████████████████████████| 11873/11873 [00:00<00:00, 20722.07it/s]


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11873 entries, 0 to 11872
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             11873 non-null  object 
 1   is_impossible  11873 non-null  int8   
 2   title          11873 non-null  object 
 3   question       11873 non-null  object 
 4   answer_start   11873 non-null  int16  
 5   answer_end     11873 non-null  int16  
 6   answer_text    11873 non-null  object 
 7   context        11873 non-null  object 
 8   cq_length      11873 non-null  int16  
 9   cq_wp_length   11873 non-null  int16  
 10  cq_sp_length   11873 non-null  int16  
 11  cq_bpe_length  11873 non-null  int16  
 12  pred           11873 non-null  object 
 13  exact_match    11873 non-null  int8   
 14  f1             11873 non-null  float32
dtypes: float32(1), int16(6), int8(2), object(6)
memory usage: 765.4+ KB


In [19]:
%%time
cols = ["id", "exact_match", "f1", "is_impossible", "title", "question", "answer_text", "pred", "context"]
df[cols].to_parquet(f"{job_dir}/{ts}.parquet", index=False)

Wall time: 40.1 ms


In [20]:
df[cols].sample(30).head(30)

,id,exact_match,f1,is_impossible,title,question,answer_text,pred,context
6933,57287b322ca10214002da3c1,0,0.000000,0,Economic_inequality,In what sector are jobs beginning to decrease?,manufacturing,"known as the kuznets curve. kuznets demonstrated this relationship using cross - sectional data. however, more recent testing of this theory with superior panel data has shown it to be very weak. kuznets'curve predicts that income inequality will eventually decrease given time. as an example, income inequality did fall in the united states during its high school movement from 1910 to 1940 and thereafter. [ citation needed ] however, recent data shows that the level of income inequality began to rise after the 1970s. this does not","Plotting the relationship between level of income and inequality, Kuznets saw middle-income developing economies level of inequality bulging out to form what is now known as the Kuznets curve. Kuznets demonstrated this relationship using cross-sectional data. However, more recent testing of this theory with superior panel data has shown it to be very weak. Kuznets' curve predicts that income inequality will eventually decrease given time. As an example, income inequality did fall in the United States during its High school movement from 1910 to 1940 and thereafter.[citation needed] However, recent data shows that the level of income inequality began to rise after the 1970s. This does not necessarily disprove Kuznets' theory.[citation needed] It may be possible that another Kuznets' cycle is occurring, specifically the move from the manufacturing sector to the service sector.[citation needed] This implies that it may be possible for multiple Kuznets' cycles to be in effect at any given time."
11813,5ad286fdd7d075001a429903,1,1.000000,1,Force,Whose mechanics affected three dimensional objects?,Newtonian,,"Newton's laws and Newtonian mechanics in general were first developed to describe how forces affect idealized point particles rather than three-dimensional objects. However, in real life, matter has extended structure and forces that act on one part of an object might affect other parts of an object. For situations where lattice holding together the atoms in an object is able to flow, contract, expand, or otherwise change shape, the theories of continuum mechanics describe the way forces affect the material. For example, in extended fluids, differences in pressure result in forces being directed along the pressure gradients as follows:"
5429,5726f96ddd62a815002e9699,0,0.000000,0,Pharmacy,Who pays Australian pharmacists for doing Home Medicines Reviews?,the Australian Government,"such as medications reviews ( medschecks in ontario ). in the united kingdom, pharmacists who undertake additional training are obtaining prescribing rights and this","This shift has already commenced in some countries; for instance, pharmacists in Australia receive remuneration from the Australian Government for conducting comprehensive Home Medicines Reviews. In Canada, pharmacists in certain provinces have limited prescribing rights (as in Alberta and British Columbia) or are remunerated by their provincial government for expanded services such as medications reviews (Medschecks in Ontario). In the United Kingdom, pharmacists who undertake additional training are obtaining prescribing rights and this is because of pharmacy education. They are also being paid for by the government for medicine use reviews. In Scotland the pharmacist can write prescriptions for Scottish registered patients of their regular medications, for the majority of drugs, except for controlled drugs, when the patient is unable to see their doctor, as could happen if they are away from home or the doctor is unavailable. In the United States, pharmaceutical care or clinical pharmacy has had an evolving influence on the practice of pharmacy. Moreover, the Doctor of Pharmacy (Pharm. D.) degree is now required before entering practice and some pharmacists now co

In [21]:
index = df[(df["is_impossible"] == 0) & (df["f1"] > 0)].index
df.iloc[index][cols].sample(30).head(30)

,id,exact_match,f1,is_impossible,title,question,answer_text,pred,context
8087,5728fc2eaf94a219006a9ec8,0,0.195122,0,Immune_system,What enzymes in saliva are antibacterial in nature?,lysozyme and phospholipase A2,"- defensins. enzymes such as lysozyme and phospholipase a2 in saliva, tears, and breast milk are also antibacterials. vaginal secretions serve as a chemical barrier following menarche, when they become slightly acidic, while semen contains defensins and zinc to","Chemical barriers also protect against infection. The skin and respiratory tract secrete antimicrobial peptides such as the β-defensins. Enzymes such as lysozyme and phospholipase A2 in saliva, tears, and breast milk are also antibacterials. Vaginal secretions serve as a chemical barrier following menarche, when they become slightly acidic, while semen contains defensins and zinc to kill pathogens. In the stomach, gastric acid and proteases serve as powerful chemical defenses against ingested pathogens."
279,56e19724cd28a01900c679f6,0,0.222222,0,Computational_complexity_theory,A function problem is an example of what?,a computational problem,"of a decision problem, that is, it isn't","A function problem is a computational problem where a single output (of a total function) is expected for every input, but the output is more complex than that of a decision problem, that is, it isn't just yes or no. Notable examples include the traveling salesman problem and the integer factorization problem."
10507,5730ac782461fd1900a9cf74,0,0.200000,0,Imperialism,When was Otto von Bismarck born?,1862,"long - time chancellor, otto von bismarck ( 1862 – 90 ),","Not a maritime power, and not a nation-state, as it would eventually become, Germany’s participation in Western imperialism was negligible until the late 19th century. The participation of Austria was primarily as a result of Habsburg control of the First Empire, the Spanish throne, and other royal houses.[further explanation needed] After the defeat of Napoleon, who caused the dissolution of that Holy Roman Empire, Prussia and the German states continued to stand aloof from imperialism, preferring to manipulate the European system through the Concert of Europe. After Prussia unified the other states into the second German Empire after the Franco-German War, its long-time Chancellor, Otto von Bismarck (1862–90), long opposed colonial acquisitions, arguing that the burden of obtaining, maintaining, and defending such possessions would outweigh any potential benefits. He felt that colonies did not pay for themselves, that the German bureaucratic system would not work well in the tropics and the diplomatic disputes over colonies would distract Germany from its central interest, Europe itself."
6852,5729e2b76aef0514001550cf,0,0.060606,0,Economic_inequality,What did the increase in skilled workers lead to?,a decrease in the price of skilled labor,"of the economy, however educational inequality in gender also influence towards the economy. lagerlof and galor stated that gender inequality in education can result to low economic growth, and continued gender inequality in education, thus creating a poverty trap. it is suggested that a large gap in male and female education may indicate backwardness and so may be associated with lower economic growth, which","During the mass high school education movement from 1910–1940, there was an increase in skilled workers, which led to a decrease in the price of skilled labor. High school education during the period was designed to equip students with necessary skill sets to be able to perform at work. In fact, it differs from the present high school education, which is regarded as a stepping-stone to acquire college and advanced degrees. This decrease in wages caused a period of compression and decreased inequality between skilled and unskilled workers. Education is very important for the growth of the economy, however educational inequality in gender also influence towards the economy. Lag